In [ ]:
from parameters import handle_continuous_param, Normal, FrequencyNoise
import imgaug.dtypes as iadt
import imgaug.parameters as iaps
import random
import imgaug as ia
#import cupyx.scipy.ndimage
from cupyimg.skimage.transform import resize

In [ ]:
def _assert_two_or_three_dims(shape):
    if hasattr(shape, "shape"):
        shape = shape.shape
    assert len(shape) in [2, 3], (
        "Expected image with two or three dimensions, but got %d dimensions "
        "and shape %s." % (len(shape), shape))

def imresize_single_image(image, sizes, interpolation=None):
    _assert_two_or_three_dims(image)

    grayscale = False
    if image.ndim == 2:
        grayscale = True
        image = image[:, :, cp.newaxis]

    rs = imresize_many_images(
        image[cp.newaxis, :, :, :], sizes, interpolation=interpolation)
    if grayscale:
        return rs[0, :, :, 0]
    return rs[0, ...]

In [ ]:
def draw_sample(random_state=None):
    return draw_samples(1, random_state=random_state)[0]

In [ ]:
def draw_on_image(image, random_state, args):
    image = cp.asarray(image)
    alpha, intensity = generate_maps(image, random_state, args)
    alpha = alpha[..., cp.newaxis]
    intensity = intensity[..., cp.newaxis]

    if image.dtype.kind == "f":
        intensity = intensity.astype(image.dtype)
        return (1 - alpha) * image + alpha * intensity

    intensity = cp.clip(intensity, 0, 255)
    # TODO use blend_alpha_() here
    return cp.clip(
        (1 - alpha) * image.astype(alpha.dtype)
        + alpha * intensity.astype(alpha.dtype),
        0,
        255
    ).astype(np.uint8)
    
def generate_maps(image, random_state, args):
    intensity_mean_sample = random.randrange(args[0][0], args[0][1])
    alpha_min_sample = 0
    alpha_multiplier_sample = random.uniform(args[4][0], args[4][1])
    alpha_size_px_max = args[5]
    intensity_freq_exponent = args[1]
    alpha_freq_exponent = args[6]
    sparsity_sample = random.uniform(args[7][0], args[7][1])
    density_multiplier_sample = random.uniform(args[8][0], args[8][1])

    height, width = image.shape[0:2]
    rss_alpha, rss_intensity = random.sample(range(0, 10), 2)

    intensity_coarse = _generate_intensity_map_coarse(
        height, width, intensity_mean_sample,
        iaps.Normal(0, scale=10),
        rss_intensity)
    
    intensity_fine = _generate_intensity_map_fine(
        height, width, intensity_mean_sample, intensity_freq_exponent,
        rss_intensity)
    intensity = intensity_coarse + intensity_fine

    alpha = _generate_alpha_mask(
        height, width, alpha_min_sample, alpha_multiplier_sample,
        alpha_freq_exponent, alpha_size_px_max, sparsity_sample,
        density_multiplier_sample, rss_alpha)

    return alpha, intensity

def _generate_intensity_map_coarse(height, width, intensity_mean,
                                       intensity_local_offset, random_state):
    height_intensity, width_intensity = (8, 8)
    intensity = cp.asarray(
        intensity_mean
        + intensity_local_offset.draw_samples(
            (height_intensity, width_intensity), random_state)
    )
    #intensity = cv2.resize(intensity,(width, height), interpolation = cv2.INTER_CUBIC)
    intensity = resize(intensity, (width, height))

    return intensity

def _generate_intensity_map_fine(height, width, intensity_mean,
                                 exponent, random_state):
    intensity_details_generator = iaps.FrequencyNoise(
        exponent=exponent,
        size_px_max=max(height, width, 1),  # 1 here for case H, W being 0
        upscale_method="cubic"
    )
    intensity_details = cp.asarray(intensity_details_generator.draw_samples(
        (height, width), random_state))
    return intensity_mean * ((2*intensity_details - 1.0)/5.0)

def _generate_alpha_mask(height, width, alpha_min, alpha_multiplier,
                         exponent, alpha_size_px_max, sparsity,
                         density_multiplier, random_state):
    alpha_generator = iaps.FrequencyNoise(
        exponent=exponent,
        size_px_max=alpha_size_px_max,
        upscale_method="cubic"
    )
    alpha_local = cp.asarray(alpha_generator.draw_samples(
        (height, width), random_state))
    alpha = alpha_min + (alpha_multiplier * alpha_local)
    alpha = (alpha ** sparsity) * density_multiplier
    alpha = cp.clip(alpha, 0.0, 1.0)

    return alpha

def clouds(img):
    temp = draw_on_image(img, 0, args1)
    return np.uint8(cp.asnumpy(draw_on_image(temp, 0, args2)))

In [ ]:
args1 = [(196, 255),
        (-2.5, -2.0),
        10,
        0,
        (0.25, 0.75),
        (2, 8),
        (-2.5, -2.0),
        (0.8, 1.0),
        (0.5, 1.0)]

args2 = [(196, 255),
            (-2.0, -1.0),
            10,
            0,
            (0.5, 1.0),
            (64, 128),
            (-2.0, -1.0),
            (1.0, 1.4),
            (0.8, 1.5)]

out = clouds(np.zeros((512,512,3)))

cv2.imshow('0',out)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# class Clouds():

#     def __init__(self,
#                  seed=None, name=None,
#                  random_state="deprecated", deterministic="deprecated"):
#         layers = [
#             CloudLayer(
#                 intensity_mean=(196, 255),
#                 intensity_freq_exponent=(-2.5, -2.0),
#                 intensity_coarse_scale=10,
#                 alpha_min=0,
#                 alpha_multiplier=(0.25, 0.75),
#                 alpha_size_px_max=(2, 8),
#                 alpha_freq_exponent=(-2.5, -2.0),
#                 sparsity=(0.8, 1.0),
#                 density_multiplier=(0.5, 1.0),
#                 seed=None,
#                 random_state=random_state,
#                 deterministic=deterministic
#             ),
#             CloudLayer(
#                 intensity_mean=(196, 255),
#                 intensity_freq_exponent=(-2.0, -1.0),
#                 intensity_coarse_scale=10,
#                 alpha_min=0,
#                 alpha_multiplier=(0.5, 1.0),
#                 alpha_size_px_max=(64, 128),
#                 alpha_freq_exponent=(-2.0, -1.0),
#                 sparsity=(1.0, 1.4),
#                 density_multiplier=(0.8, 1.5),
#                 seed=None,
#                 random_state=random_state,
#                 deterministic=deterministic
#             )
#         ]
        
#         super(Clouds, self).__init__(
#             (1, 2),
#             children=layers,
#             random_order=False,
#             seed=seed, name=name,
#             random_state=None, deterministic=False)